# Implement a Decision Tree

*CS 334 - Algorithms of Machine Learning | Conrad Kennington*

*Computer Science | Boise State University*

*11.02.2022 | Fall 2022*

*Aida Gomezbueno Berezo | aidagomezbuenobe@u.boisestate.edu*

*The decision tree algorithim is learning from Congressional Voting Records dataset, trying to predict party affiliation (Democrat/Republican) from 1984 Congressional Voting Records, while performing **Information Gain** splitting strategy.*

>*This [data set](https://www.kaggle.com/datasets/devvret/congressional-voting-records) includes votes for each of the U.S. House of Representatives Congressmen on the 16 key votes identified by the CQA. The CQA lists nine different types of votes: voted for, paired for, and announced for (these three simplified to yea), voted against, paired against, and announced against (these three simplified to nay), voted present, voted present to avoid conflict of interest, and did not vote or otherwise make a position known (these three simplified to an unknown disposition).*


In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

#### Download | Split dataset

In [2]:
#Download
house_votes = pd.read_csv("house-votes-84.csv")

#Set columns in order to be able to refer to them
cols = ['Class Name', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa']
house_votes.columns = cols
feature_names = ['handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution','physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending','superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']
label = ['Class-Name']

#Get X and Y: features and label to train the model, then testing
X = house_votes[feature_names].values
Y = house_votes['Class Name'].values.reshape(-1,1)

#Spliting dataset into training/testing sets
X_train,X_test,y_train,y_test=train_test_split(X, Y, test_size=0.30, random_state=100)

#### Object Oriented Fashion | Node & Decision Tree

In [3]:
class Node():
    def __init__(self, feature_index=None, left=None, right=None, threshold=None, info_gain=None, leaf_value=None):
        self.feature_index = feature_index
        self.left = left
        self.right = right
        self.threshold = threshold
        self.info_gain = info_gain
        self.leaf_value = leaf_value

class DecisionTree():
    
    '''
        Decision Tree constructor. Root: initialized as None, min_splits: to set min number of splits per split, max_depth: to set how many levels of splits do we want.
    '''   
    def __init__(self,min_splits=2, max_depth=2):
        self.root = None
        self.min_splits = min_splits
        self.max_depth = max_depth
     
    '''
        Build tree with depth 0.
    '''
    def grow_tree(self, data, depth=0):
        X, Y = data[:,:-1],data[:,-1] 
        num_samples, num_features = np.shape(X)
        #Consider bounds of tree: min_splits and max_depth
        if num_samples >= self.min_splits and depth <= self.max_depth:
            best_split = self.find_best_split(data, num_samples, num_features)
            if best_split['info_gain'] > 0:
                left_tree = self.grow_tree(best_split['subtree_left'], depth + 1)
                right_tree = self.grow_tree(best_split['subtree_right'], depth + 1)
                return Node(best_split['feature_index'], left_tree, right_tree, best_split['threshold'], best_split['info_gain'])
        leaf_list = list(Y)
        leaf_val = max(leaf_list, key=leaf_list.count)
        return Node(leaf_value=leaf_val)
    
    '''
        Run through all the features and its values in order to find the thresold that results in the best split and, thus, build the most accurate tree.
    '''
    def find_best_split(self, data, num_samples, num_features):
        split_d = {}
        best_gain = -1 
        for feat in range(num_features):
            vals = data[:feat]
            unique_vals = np.unique(vals)
            for threshold in unique_vals:
                subtree_l, subtree_r = self.split(data, feat, threshold)
                if len(subtree_l) > 0 and len(subtree_r) > 0:
                    parent = data[:,-1]
                    child_l = subtree_l[:,-1]
                    child_r = subtree_r[:,-1]
                    ig = self.information_gain(parent, child_l, child_r)
                    if ig > best_gain: 
                        split_d['feature_index'] = feat
                        split_d['threshold'] = threshold
                        split_d['info_gain'] = ig
                        split_d['subtree_left'] = subtree_l
                        split_d['subtree_right'] = subtree_r
                        best_gain = ig 
        return split_d
    
    '''
        From the threshold previously found, split the data.
    '''
    def split(self, data, i, threshold):
        trues = []
        falses = []
        for row in data:
            if row[i] <= threshold:
                trues.append(row)
            else:
                falses.append(row)
        subtree_l = np.array(trues)
        subtree_r = np.array(falses)
        return subtree_l, subtree_r
    
    '''
        From the entropy of both, parent and (average of) child nodes, Information Gain is computed to determine how good the splitting of nodes in a decision tree are, and thus, split (find_best_split, split).
    '''
    def information_gain(self, parent, child_l, child_r):
        weight_l = len(child_l) / len(parent)
        weight_r = len(child_r) / len(parent)        
        children = (weight_l * self.entropy_k(child_l)) + (weight_r * self.entropy_k(child_r))
        ig = self.entropy_k(parent) - children #parent_entropy - child_entropy    
        return ig
    
    '''
        Used by Information Gain as a theory metric to measure the impurity or uncertainty in a group of observations (either nodes). 
    '''
    def entropy_k(self, parent):
        labels = np.unique(parent)
        entropy_k = 1
        for label in labels: #democrat vs republican
            num_rows_true = parent[parent == label]
            p = len(num_rows_true) / len(parent)
            entropy_k = entropy_k - p**2 #exponentiation  
        return entropy_k
    
    '''
        It builds the decision tree from the given training set (X, Y). 
    '''
    def _fit(self, X, Y):
        data = np.concatenate((X, Y), axis=1) 
        self.root = self.grow_tree(data)
    
    '''
        It gathers all the predictions by calling get_predictions. Then, returning an array with all the results.
    '''
    def predict(self, X):
        results = []
        for row in X:
            results.append(self.get_predictions(self.root, row))
        return pd.array(results)
    
    '''
        It gets the final predictions (results) of the decision tree model.
        It recursevily runs through all the tree until it reaches a leaf node, then returning it.
    '''
    def get_predictions(self, tree, x):
        if tree.leaf_value is not None:
            return tree.leaf_value
        feat_id = x[tree.feature_index]
        if feat_id <= tree.threshold:
            return self.get_predictions(tree.left, x)
        return self.get_predictions(tree.right, x)

#### Testing | Error rate

In [4]:
tree = DecisionTree(1000, 5)
tree._fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("Accuracy: %.2f%% " % (accuracy_score(y_test, y_pred)*100))

Accuracy: 65.65% 


In [5]:
tree = DecisionTree(100, 5)
tree._fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("Accuracy: %.2f%% " % (accuracy_score(y_test, y_pred)*100))

Accuracy: 93.13% 


In [6]:
tree = DecisionTree(2, 5)
tree._fit(X_train, y_train)
#tree.print_results()
y_pred = tree.predict(X_test)
print("Accuracy: %.2f%% " % (accuracy_score(y_test, y_pred)*100))

Accuracy: 93.89% 


In [42]:
#Confusion matrix
cf = confusion_matrix(y_test, y_pred)
tp = cf[0][0]
fp = cf[0][1]
fn = cf[1][0]
tn = cf[1][1]
#Accuracy
acc = (tp+tn)/(tp+tn+fp+fn)
#Precision
prec = tp/(tp+fp)
#Recall/Sensitivity
recall = tp/(tp+tn)
#F1 score
f1 = 2*((prec*recall)/(prec+recall))
#Error rate = 1 - accuracy
error_rate = 1-acc
n_errors = fn+fp

#Print results
print("Confusion matrix: ")
for i in cf:
    print("\t\t", i)
print("Accuracy: %.2f%%" % (acc*100))
print("Precision: %.2f%%" % (prec*100))
print("Recall/Sensitivity: %.2f%%" % (recall*100))
print("F1-Score: %.2f%%" % (f1*100))
print("Error rate: %.2f%%" % (error_rate*100))
print("Specific number of error: ", n_errors)

Confusion matrix: 
		 [81  5]
		 [ 3 42]
Accuracy: 93.89%
Precision: 94.19%
Recall/Sensitivity: 65.85%
F1-Score: 77.51%
Error rate: 6.11%
Specific number of error:  8
